## 라이브러리 import

In [22]:
import pandas as pd
import numpy as np
import pickle

from eunjeon import Mecab
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences


## 데이터 로드

In [2]:
# dataframe print 이쁘게
pd.set_option('display.unicode.east_asian_width', True)

In [3]:
df = pd.read_csv('./data/raw_65548_2021-06-18.csv', index_col=0).reset_index()
print(df)
print(df.info())

                             title  \
0        황성주 박사의 생식과 건강   
1      아름다운 청년, 대니 서의 집   
2               여왕 패션 종이인형   
3           더 포스터 북 by 이미소   
4                        가죽 소품   
...                            ...   
65543            이성적 낙관주의자   
65544          현산어보를 찾아서 1   
65545              전염병의 문화사   
65546                    서울 해법   
65547  세상을 이루는 모든 원소 118   

                                                                                     summary  \
0      생식을 통한 식생활 개선으로 성인병을 치료하는 방법서. 육류 등의 포화지방이나 콜레...    
1      환경 운동가 대니 서가 공개하는 환경 친화적인 집 꾸미기. 이 책은 교외에 위치한 ...       
2      다양한 업적을 가진 세계의 여왕 패션을 직접 입혀보고, 아름답고 다양한 드레스와 왕...     
3      직관적인 제목처럼 권마다 포스터 작품 10점으로 가득 채워진 도서이다. 그러나 일반...      
4      가죽공예의 기본 테크닉에서부터 명함 지갑, 동전 지갑, 물림쇠 지갑, 파우치, 안경...       
...                                                                                  ...       
65543  <게놈>, <붉은 여왕>의 세계적 과학저술가 매트 리들리 최신작. 진화심리, 생명과...         
65544  --한국과학문화재단 '우수과학인증도서', 「시사저널」 '올해(2003년)의 책

## 전처리

### 중복값 및 결측값 제거

In [4]:
# 중복값 제거
df.drop_duplicates(subset=['title'], inplace=True)
print(df['title'].duplicated().sum())

0


In [6]:
# 결측값 제거
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

### feature, target 분리

In [7]:
X = df['title'] + " " + df['summary']
Y = df[['category']]

### target encoding 및 encoder 저장

In [8]:
encoder = OneHotEncoder(sparse=False)
onehot_Y = encoder.fit_transform(Y)
label = encoder.categories_
print(label)
print(onehot_Y)

[array(['Art_Culture', 'Economic', 'Economic_Management',
       'Health_Hobby_Leisure', 'Home_Cook_Beauty', 'Novel_Poem',
       'Reference_Book', 'Religion_Mysticism'], dtype=object)]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [9]:
# 나중에 다른 문장을 토큰화할때 동일한 classes_로 인코딩해야함 -> 학습시킨 encoder 저장
# pickle로 저장 / 불러오기시 원본의 데이터 타입 그대로 유지
with open('./data/nouns_category_onehot_encoder.pickle', 'wb') as f:
    pickle.dump(encoder, f)

## 형태소 분석

In [10]:
# 데이터 형태소 분석
m = Mecab()
for i in range(len(X)):
    X[i] = m.nouns(X[i])
print(X)

0        [황성주, 박사, 생식, 건강, 생식, 식생활, 개선, 성인병, 치료, 방법, 육류...
1        [청년, 집, 환경, 운동가, 공개, 환경, 친화, 집, 책, 교외, 위치, 부모,...  
2        [여왕, 패션, 종이, 인형, 다양, 업적, 세계, 여왕, 패션, 드레스, 왕관, ... 
3        [포스터, 북, 미소, 직관, 제목, 권, 포스터, 작품, 점, 도서, 일반, 종이... 
4        [가죽, 소품, 가죽, 공예, 기본, 테크닉, 명함, 지갑, 동전, 지갑, 물림쇠,...
                                           ...                                    
58381    [파브르, 곤충기, 년, 완결, 파브르, 곤충기, 권, 자랑, 책, 곤충, 연구, ... 
58382    [자동, 제어, 공학, 빛, 교재, 시리즈, 권, 자동, 제어, 공학, 공학, 계열... 
58383    [현산어보, 한국과학문화재단, 우수, 과학, 인증, 도서, 시사저널, 올해, 년, ...
58384    [전염병, 문화사, 인간, 생명체, 미생물, 인간, 병원, 미생물, 역사, 관계, ...
58385    [세상, 원소, 물질, 기본, 원소, 직관, 이해, 수, 비주얼, 백, 책, 지금,...  
Length: 58386, dtype: object


In [11]:
# stopword 로드
stopwords = pd.read_csv('../datasets/stopwords.csv', index_col=0)
print(stopwords)

    stopword
0         아
1         휴
2     아이구
3     아이쿠
4     아이고
..       ...
783   없다는
784   않을까
785     있지
786   있으며
787   그대로

[787 rows x 1 columns]


In [12]:
# 함수로 만들어 기존 데이터에 apply해 stopword 제거
def delete_stopwords(lst):
    words = []
    for word in lst:
        if word not in list(stopwords['stopword']) and len(word) > 1:
            words.append(word)
    return ' '.join(words)

X = X.apply(delete_stopwords)

## 토큰화 및 저장

In [13]:
# stopword 제거한 뉴스 타이틀 토큰화
token = Tokenizer()
token.fit_on_texts(X)
tokened_X = token.texts_to_sequences(X)
print(tokened_X[0])

[17307, 319, 3289, 45, 3289, 1585, 787, 4422, 157, 5, 3018, 8331, 866, 4378, 2087, 4423, 63, 45, 22761, 458, 514, 3289, 808, 214, 78]


In [14]:
# 단어의 개수 출력 / padding한 0을 포함하기 위해 +1
wordsize = len(token.word_index) + 1  # word_index는 토큰화한 단어와 숫자 보여줌 / 0을 포함하지 않고 1부터 시작
print(wordsize)

68561


In [15]:
with open('./data/nouns_token.pickle', 'wb') as f:
    pickle.dump(token, f)

## 문장 길이 맞춤

In [16]:
# 형태소가 가장 많은 문장의 단어 수
max = 0
for i in range(len(tokened_X)):
    if max < len(tokened_X[i]):
        max = len(tokened_X[i])
print(max)

612


In [23]:
# 길이가 맞게 앞쪽에 0 붙여줌
X_pad = pad_sequences(tokened_X, max)
print(X_pad[:10])

[[   0    0    0 ...  808  214   78]
 [   0    0    0 ...   86    1  323]
 [   0    0    0 ...   96 1987 2513]
 ...
 [   0    0    0 ... 1929  219 2010]
 [   0    0    0 ...   12  117  260]
 [   0    0    0 ...  979 5462  442]]


## 데이터 저장

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_pad, onehot_Y, test_size = 0.1
)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(52547, 612)
(5839, 612)
(52547, 8)
(5839, 8)


In [25]:
xy = X_train, X_test, Y_train, Y_test
np.save(f'./data/nouns_books_data_max_{max}_size_{wordsize}.npy', xy)

C:\Users\ASIA-23\anaconda3\envs\AI_exam\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
